# Рекомендация тарифов для пользователей

[Изучение общей информации](#Изучение-общей-информации)

[Деление данных на выборки](#Деление-данных-на-выборки)

[Исследование моделей](#Исследование-моделей)

[Проверка моделей на тестовой выборке](#Проверка-моделей-на-тестовой-выборке)

[Проверка модели на адекватность](#Проверка-модели-на-адекватность)


# Изучение общей информации

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv('/datasets/users_behavior.csv')
print(df)
print(df.info())
print('==============================')

def info(df): # Функция пробегает по признакам датафрейма и оценивает распределение

    for column in df.columns:       
            import plotly.express as px
            print(df[column].describe())
            fig = px.histogram(df, x = column, marginal = 'box', title = 'Распределение '+ column)
            fig.show()
            
#info(df)

      calls  minutes  messages   mb_used  is_ultra
0      40.0   311.90      83.0  19915.42         0
1      85.0   516.75      56.0  22696.96         0
2      77.0   467.66      86.0  21060.45         0
3     106.0   745.53      81.0   8437.39         1
4      66.0   418.74       1.0  14502.75         0
...     ...      ...       ...       ...       ...
3209  122.0   910.98      20.0  35124.90         1
3210   25.0   190.36       0.0   3275.61         0
3211   97.0   634.44      70.0  13974.06         0
3212   64.0   462.32      90.0  31239.78         0
3213   80.0   566.09       6.0  29480.52         1

[3214 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


### Вывод

Таблица открыта и изучена. Таблица имеет 3214 строк и 5 столбцов. Использование метода describe() и построение гистограмм позволило нам оценить распределение некоторых признаков и увидеть аномалии.

# Деление данных на выборки

In [2]:
df_train, df_temp = train_test_split(df, test_size = 0.4, random_state=12345) # Разбили исходный df на промежуточно_тестовую 60% и промежуточно_валидационную 40%
df_valid, df_test = train_test_split(df_temp, test_size = 0.5, random_state=12345) # Разбили промежуточно_валидационную на тестовую и валидационную 

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Исследование моделей

In [3]:
# Дерево решений
from sklearn.metrics import accuracy_score

result_tree = 0
tree_depth = 0

for depth in range(1, 10):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # инициализировали модель DecisionTreeClassifier с параметром random_state=12345 и max_depth=depth
    model_tree.fit(features_train, target_train) # обучите модель на тренировочной выборке
    predictions_valid = model_tree.predict(features_valid) # получите предсказания модели на валидационной выборке
    accuracy_tree = accuracy_score(target_valid, predictions_valid)
    #result = mean_squared_error(target_valid, predictions_valid)**0.5 # посчитайте значение метрики rmse на валидационной выборке
    if accuracy_tree > result_tree:
        result_tree = accuracy_tree
        tree_depth = depth

print("Глубина дерева:", tree_depth)
print(result_tree)

Глубина дерева: 3
0.7853810264385692


In [4]:
# Cлучайный лес

result_forest = 0
estimators = 0

for n in range(1, 11):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators = n, min_samples_leaf = n) # инициализируем модель с заданным количеством деревьев
    model_forest.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model_forest.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
#     predictions_valid = model_forest.predict(features_valid) # получите предсказания модели на валидационной выборке
#     accuracy_forest = accuracy_score(target_valid, predictions_valid)
    if result > result_forest:
        result_forest = result #  сохраните наилучшее значение метрики accuracy на валидационных данных
        estimators = n
        
print("Количество деревьев:", estimators)
print("Accuracy наилучшей модели на валидационной выборке:", result_forest)

Количество деревьев: 6
Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751


In [5]:
# Логистическая регрессия
model_regression = LogisticRegression(solver='lbfgs', random_state=12345) # инициализируем модель LogisticRegression
model_regression.fit(features_train, target_train) # обучите модель на тренировочной выборке
result_regression = model_regression.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
print(result_regression)

0.7107309486780715


# Проверка моделей на тестовой выборке

In [6]:
# Дерево решений
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=3)
model_tree.fit(features_train, target_train)
predictions_tree_test = model_tree.predict(features_test)
accuracy_tree = accuracy_score(target_test, predictions_tree_test)
print(accuracy_tree)

0.7791601866251944


In [7]:
# Cлучайный лес
model_forest = RandomForestClassifier(random_state=12345, n_estimators = 6, min_samples_leaf = 6)
model_forest.fit(features_train, target_train)
predictions_forest_test = model_forest.predict(features_test)
accuracy_forest = accuracy_score(target_test, predictions_forest_test)
print(accuracy_forest)

0.7931570762052877


In [8]:
# Логистическая регрессия
model_regression = LogisticRegression(solver='lbfgs', random_state=12345) # инициализируем модель LogisticRegression
model_regression.fit(features_train, target_train) # обучите модель на тренировочной выборке
model_regression_test = model_regression.predict(features_test)
result_regression = accuracy_score(target_test, model_regression_test)
print(result_regression)

0.6842923794712286


## Вывод

Согласно проведенным исследованием и тестированию на тестовой выборке данных модель `случайный лес` с показателем качества `0.7931570762052877` является лучшей.

# Проверка модели на адекватность

In [9]:
from sklearn.dummy import DummyClassifier
model_forest = RandomForestClassifier(random_state=12345, n_estimators = 6, min_samples_leaf = 6)
model_forest.fit(features_train, target_train)
predictions_forest_test = model_forest.predict(features_test)
accuracy_forest = accuracy_score(target_test, predictions_forest_test)
print('Качество модели:', accuracy_forest)

model_dummy = DummyClassifier(strategy='uniform', random_state=0)
model_dummy.fit(features_train, target_train)
predictions_dummy_test = model_dummy.predict(features_test)
dummy_score = accuracy_score(target_test, predictions_dummy_test)
print('Качество рандомной модели:', dummy_score)


Качество модели: 0.7931570762052877
Качество рандомной модели: 0.5303265940902022


Качество нашей лучшей модели выше, чем у рандомной модели. Это позволяет нам думать, что она адекватна.